In [1]:
import pandas as pd

In [11]:
df_基础 = pd.read_csv('../data/现代线粒体DNA汇总.csv').loc[:,['ID','Sequence_method']]
df_基础 = df_基础.loc[df_基础['Sequence_method']=='WHOLE']
df_质量控制 = pd.read_csv('../data/质量控制.csv').iloc[:,0:4]
df_质量控制QC = df_质量控制.loc[df_质量控制['Quality']>=0.95]
df_质量控制QC = df_质量控制QC.merge(df_基础,left_on='SampleID',right_on='ID',how='left')
df_质量控制QC.dropna(how='any',inplace=True)
df_质量控制QC

,SampleID,Haplogroup,Rank,Quality,ID,Sequence_method
0,KU682980,I2,1,1.0092,KU682980,WHOLE
1,KU683240,I2,1,1.0092,KU683240,WHOLE
2,KU683033,I2,1,1.0092,KU683033,WHOLE
3,KU683502,I2,1,1.0092,KU683502,WHOLE
4,EF556173,L5a1a,1,1.0080,EF556173,WHOLE
...,...,...,...,...,...,...
39663,KX456745,N21+195,1,0.9500,KX456745,WHOLE
39664,SD244,N9a2'4'5'11,1,0.9500,SD244,WHOLE
39665,MN849792,Q1,1,0.9500,MN849792,WHOLE
39667,111-1916-8482,U3a,1,0.9500,111-1916-8482,WHOLE


In [13]:
# 随机选择200个样本，要求Haplogroup不相同，且Haplogroup列要有至少1个以L开头的样本
# 1. 每个Haplogroup保留一个样本
df_unique = df_质量控制QC.drop_duplicates(subset=['Haplogroup'])


In [19]:
# 2. 找出以L开头的Haplogroup样本
df_L = df_unique[df_unique['Haplogroup'].str.startswith('L')]

if df_L.empty:
    raise ValueError("数据中没有以L开头的Haplogroup样本，无法满足要求！")

# 3. 从以L开头的样本中随机选取1个
sample_L = df_L.sample(1, random_state=42)

# 4. 剩余样本中去掉已经选过的Haplogroup
df_remaining = df_unique[~df_unique['Haplogroup'].isin(sample_L['Haplogroup'])]

# 5. 从剩余的样本中随机选择(200-1)个
if len(df_remaining) < 199:
    raise ValueError("可选的独立Haplogroup数量不足，无法抽取199个样本！")

sample_rest = df_remaining.sample(199, random_state=42)

# 6. 合并最终的200个样本
df_final = pd.concat([sample_L, sample_rest]).reset_index(drop=True)

# 结果
df_final.sort_values(by=['Haplogroup']).loc[:,['ID','Haplogroup']].to_csv('../data/Sample_Hap_list.txt',sep='\t',index=False,header=None)
df_final.sort_values(by=['Haplogroup']).loc[:,['ID']].to_csv('../data/Sample_list.txt',sep='\t',index=False,header=None)

In [21]:
!bcftools view -S /mnt/f/0_现代DNA处理流程/script/2_ML_MRCA/data/Sample_list.txt \
    -Oz -o ../data/Example.vcf.gz \
    /mnt/f/0_现代DNA处理流程/output/Merge_VCF/merged_biallelic_AGCT_SNP_Annotated_FULL_去除hots.vcf.gz

In [22]:
!bcftools index --threads 16 ../data/Example.vcf.gz

In [2]:

!/mnt/e/Scientifc_software/iqtree-3.0.0-Linux-intel/bin/iqtree3 \
  -redo \
  -m HKY \
  -s ../data/Example_rename.fasta \
  -T 16 \
  -o KC346074_L0d1b1a \
  -pre ../data//Example


IQ-TREE version 3.0.0 for Linux x86 64-bit built Apr  4 2025
Developed by Bui Quang Minh, Thomas Wong, Nhan Ly-Trong, Huaiyan Ren
Contributed by Lam-Tung Nguyen, Dominik Schrempf, Chris Bielow,
Olga Chernomor, Michael Woodhams, Diep Thi Hoang, Heiko Schmidt

Host:    GiantWhiteLin (AVX2, FMA3, 31 GB RAM)
Command: /mnt/e/Scientifc_software/iqtree-3.0.0-Linux-intel/bin/iqtree3 -redo -m HKY -s ../data/Example_rename.fasta -T 16 -o KC346074_L0d1b1a -pre ../data//Example
Seed:    242124 (Using SPRNG - Scalable Parallel Random Number Generator)
Time:    Mon Apr 28 22:07:00 2025
Kernel:  AVX+FMA - 16 threads (24 CPU cores detected)

Reading alignment file ../data/Example_rename.fasta ... Fasta format detected
Reading fasta file: done in 0.026032 secs using 54.95% CPU
Alignment most likely contains DNA/RNA sequences
Constructing alignment: done in 0.0105229 secs using 1795% CPU
Alignment has 200 sequences with 9419 columns, 700 distinct patterns
381 parsimony-informative, 534 singleton sites, 

In [1]:
!cat ../data/Example.treefile >> ../data/Example.boottrees

In [ ]:
from Bio import Phylo

# 1. 指定树文件路径
tree_file = '../data/Example.treefile'

# 2. 读取所有 bootstrap 树
trees = list(Phylo.parse(tree_file, 'newick'))
num_trees = len(trees)

# 3. 统计样本数（假设每棵树的叶子节点数一致，以第一棵树为准）
num_samples = len(trees[0].get_terminals()) if num_trees > 0 else 0

# 4. 读取原始文件内容（保留所有树的 Newick 文本）
with open(tree_file, 'r') as f:
    original_content = f.read().strip()

# 5. 将统计信息和原始内容写回同一文件
with open(tree_file, 'w') as f:
    # 首行：树的数量 和 样本数
    f.write(f" {num_samples} {num_trees}\n")
    # 后续行：原始的所有 bootstrap 树
    f.write(original_content + "\n")

print(f"已成功更新 {tree_file} ：找到 {num_trees} 棵树，样本数为 {num_samples}")


已成功更新 ../data/Example.treefile ：找到 1 棵树，样本数为 197
